<a href="https://colab.research.google.com/github/Teasotea/BioNER-and-RD/blob/main/Finetuning%20SciBERT%20with%20SpaCy%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import io
from nltk.tokenize import TreebankWordTokenizer as twt

In [17]:
from more_itertools import flatten

In [3]:
%mkdir data

mkdir: cannot create directory ‘data’: File exists


In [4]:
%cd data

/content/data


# Load Data

## Load datasets

In [7]:
# train_df = pd.read_csv('/content/cdr_dner_train_df.csv')
# test_df = pd.read_csv('/content/cdr_dner_test_df.csv')
# dev_df = pd.read_csv('/content/cdr_dner_dev_df.csv')


In [15]:
iob_train = pd.read_csv('/content/iob_train.csv')
iob_test = pd.read_csv('/content/iob_test.csv')
iob_dev = pd.read_csv('/content/iob_dev.csv')

In [16]:
iob_train = iob_train.applymap(eval)
iob_test = iob_test.applymap(eval)
iob_dev = iob_dev.applymap(eval)

In [9]:
# iob_train['tokens'] = iob_train['tokens'].apply(lambda x: x.strip('][\'').split('\', \''))
# iob_train['iob_tags'] = iob_train['iob_tags'].apply(lambda x: x.strip('][\'').split('\', \''))

# iob_test['tokens'] = iob_test['tokens'].apply(lambda x: x.strip('][\'').split('\', \''))
# iob_test['iob_tags'] = iob_test['iob_tags'].apply(lambda x: x.strip('][\'').split('\', \''))

# iob_dev['tokens'] = iob_dev['tokens'].apply(lambda x: x.strip('][\'').split('\', \''))
# iob_dev['iob_tags'] = iob_dev['iob_tags'].apply(lambda x: x.strip('][\'').split('\', \''))

## Convert to tsv

In [11]:
# def to_tsv_iob(df):
#   tokens, tags = [], []
#   for i,j in zip(df['tokens'], df['iob_tags']):  
#     for k,l in zip(i,j):
#         tokens.append(k)
#         tags.append(l)
#   iob_tags = pd.DataFrame({"tokens":tokens, "tags":tags}).set_index('tokens')
#   return iob_tags

In [19]:
# to_tsv_iob(iob_train)['tags'].value_counts()

In [10]:
# train_iob = to_tsv_iob(iob_train)
# test_iob = to_tsv_iob(iob_test)
# dev_iob = to_tsv_iob(iob_dev)

In [18]:
train_iob = iob_train.explode(["tokens", "iob_tags"]).set_index("tokens")
test_iob = iob_test.explode(["tokens", "iob_tags"]).set_index("tokens")
dev_iob = iob_dev.explode(["tokens", "iob_tags"]).set_index("tokens")

In [20]:
train_iob.to_csv('/content/data/iob_tsv_train.tsv', sep = '\t')
test_iob.to_csv('/content/data/iob_tsv_test.tsv', sep = '\t')
dev_iob.to_csv('/content/data/iob_tsv_dev.tsv', sep = '\t')

## Create .json files

In [21]:
!python -m spacy convert /content/data/iob_tsv_train.tsv ./ -t json -s -c conll
!python -m spacy convert /content/data/iob_tsv_dev.tsv ./ -t json -s -c conll
!python -m spacy convert /content/data/iob_tsv_test.tsv ./ -t json -s -c conll


ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1 documents): iob_tsv_train.json
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1 documents): iob_tsv_dev.json
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
ℹ Segmenting sentences with sentencizer. (Use `-b model` for improved
parser-based sentence segmentation.)
✔ Generated output file (1 documents): iob_tsv_test.json


## Create .spacy files

In [22]:
!python -m spacy convert /content/data/iob_tsv_dev.json ./ -t spacy
!python -m spacy convert /content/data/iob_tsv_train.json ./ -t spacy
!python -m spacy convert /content/data/iob_tsv_test.json ./ -t spacy

✔ Generated output file (878 documents): iob_tsv_dev.spacy
✔ Generated output file (855 documents): iob_tsv_train.spacy
✔ Generated output file (912 documents): iob_tsv_test.spacy


# Initializing SpaCy Pipeline

In [23]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!apt-key add /var/cuda-repo-9–2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-07-01 17:13:40--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-07-01 17:13:40--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?DRKJcNY-ARbqpiZ0-M07XWP_NYiglPkcHbxoQLMVu-4UWG0sT8MOUWb6ij903nOvbAZh9XLxztb-ebOqAdqIdHGAt5j2bCLqGN4PDL_kLCVNi

In [24]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [25]:
%cd ..

/content


In [26]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 26 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [27]:
!pip install -U spacy[cuda92,transformers]
!export CUDA_PATH=”/usr/local/cuda-9.2"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [28]:
!pip install git+https://github.com/explosion/spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/explosion/spacy-transformers to /tmp/pip-req-build-rv_7u2bp
  Running command git clone -q https://github.com/explosion/spacy-transformers /tmp/pip-req-build-rv_7u2bp


In [29]:
!python -m spacy init fill-config /content/config.cfg /content/config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [30]:
!python -m spacy debug data /content/config.cfg


============================ Data file validation ============================
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
✔ Pipeline can be initialize

In [31]:
!pip uninstall cupy
!pip install cupy-cuda100

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Finetune SciBERT with Spacy Pipeline

In [ ]:
!python -m spacy train -g 0 /content/config.cfg --output ./

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-07-01 17:15:38,964] [INFO] Set up nlp object from config
[2022-07-01 17:15:38,977] [INFO] Pipeline: ['transformer', 'ner']
[2022-07-01 17:15:38,982] [INFO] Created vocabulary
[2022-07-01 17:15:38,983] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification 